In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime
from dateutil.parser import parse
import math
from numpy import mean

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_validate, StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline as SKLpipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from dtreeviz.trees import dtreeviz 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as IMBLpipeline

from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import ipynbname
nb_fname = ipynbname.name()

pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 30)

In [2]:
# read df pickle
df_alg = pd.read_pickle("objects/df_alg-HAB_part1-preprocessing-5_1")
data = pd.read_pickle("objects/df_cons4-HAB_part1-preprocessing-5_1")

# data.set_index('date', inplace=True)
# slice by station and time
# df_cons = df_cons[df_cons["sampling station"] == "Debeli_rtic"].loc["2008-01-01" : "2021-12-31"]

data.isnull().sum()

date                       0
sampling station           0
DSP                        1
DSP_like                  60
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    661
salinity                 191
T                        244
SECCHI                   696
DIN                      542
PO4-P                    534
temp_dif                 346
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

In [3]:
data

,date,sampling station,DSP,DSP_like,Dinophysis caudata,Dinophysis fortii,Phalacroma rotundatum,Dinophysis sacculus,Dinophysis tripos,sun [h],air temp,wind strength,precipitation,Chl-a,salinity,T,SECCHI,DIN,PO4-P,temp_dif,Soca,month,lipophylic_toxins
0,1994-05-17,Seca,68.0,NaN,27.0,0.0,21.0,0.0,0.0,197.1,15.114286,2.933333,26.2,1.15,33.780000,13.580000,7.0,8.040,0.15,0.0,3030.486,5,NaN
3,1994-05-17,Strunjan,17.0,NaN,8.0,0.0,0.0,0.0,0.0,197.1,15.114286,2.933333,26.2,0.45,36.180000,12.620000,8.0,1.760,0.12,0.0,3030.486,5,NaN
6,1994-06-06,Debeli_rtic,27.0,NaN,16.0,3.0,5.0,0.0,0.0,166.3,18.790476,3.157143,18.8,1.11,35.530000,17.030000,NaN,2.620,0.22,0.0,3931.331,6,NaN
7,1994-06-13,Strunjan,23.0,NaN,3.0,0.0,0.0,0.0,0.0,179.1,18.533333,3.052381,23.9,1.05,35.010000,19.890000,NaN,3.920,0.09,0.0,4090.322,6,NaN
8,1994-06-21,Seca,20.0,NaN,8.0,0.0,0.0,0.0,0.0,157.5,19.095238,3.066667,29.8,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3221.804,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,2021-11-30,Seca,140.0,0.0,10.0,60.0,40.0,0.0,0.0,66.8,9.823810,2.738095,49.4,0.58,37.880001,15.500000,5.0,1.372,0.01,0.0,3392.723,11,poz
1648,2021-11-30,Debeli_rtic,50.0,10.0,0.0,30.0,20.0,0.0,0.0,66.8,9.823810,2.738095,49.4,0.69,37.860001,15.150000,NaN,1.145,0.01,0.0,3392.723,11,poz
1650,2021-12-14,Seca,10.0,0.0,0.0,0.0,10.0,0.0,0.0,64.4,6.371429,2.980952,114.4,0.58,37.880001,15.500000,5.0,1.372,0.01,0.0,3357.036,12,neg
1649,2021-12-14,Strunjan,60.0,0.0,0.0,20.0,30.0,0.0,0.0,64.4,6.371429,2.980952,114.4,0.46,37.930000,16.030001,6.0,1.204,0.01,0.0,3357.036,12,neg


In [4]:
# Descriptive statistics
data.describe()

,DSP,DSP_like,Dinophysis caudata,Dinophysis fortii,Phalacroma rotundatum,Dinophysis sacculus,Dinophysis tripos,sun [h],air temp,wind strength,precipitation,Chl-a,salinity,T,SECCHI,DIN,PO4-P,temp_dif,Soca,month
count,1451.000000,1392.000000,1451.000000,1451.000000,1451.000000,1451.000000,1451.000000,1452.000000,1452.000000,1452.000000,1452.000000,791.000000,1261.000000,1208.000000,756.000000,910.000000,918.000000,1106.000000,1452.000000,1452.000000
mean,105.749828,25.187500,24.917574,26.145762,14.198966,28.545210,4.504273,158.804614,16.602391,2.990217,57.578237,0.874185,37.155858,19.633252,8.061772,3.180930,0.064129,53.148578,3294.638168,7.382231
std,339.891474,117.295911,87.923656,160.058713,29.930657,177.903558,41.178956,60.634485,6.057941,0.405882,50.712645,0.754242,27.158301,5.273479,3.259124,3.827742,0.171599,62.590049,2329.615626,2.824143
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.800000,-0.823810,1.476190,0.000000,0.090000,24.130000,6.240000,0.300000,0.056000,0.000000,-17.807142,593.917000,1.000000
25%,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,112.600000,12.180952,2.742857,19.675000,0.400000,35.830000,15.800000,6.000000,0.830000,0.020000,0.000000,1636.004750,5.000000
50%,37.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,161.400000,17.442857,2.980952,44.700000,0.680000,36.870000,20.870000,7.300000,1.905000,0.040000,21.319393,2580.824000,8.000000
75%,90.000000,20.000000,13.000000,10.000000,20.000000,10.000000,0.000000,207.300000,21.738095,3.214286,79.000000,1.095000,37.460000,23.920000,10.000000,3.822750,0.070000,105.747809,4236.820500,10.000000
max,7630.000000,2920.000000,1309.000000,4624.000000,393.000000,4639.000000,1139.000000,277.800000,26.566667,5.257143,267.700000,9.250000,999.000000,28.370001,20.000000,35.470000,3.540000,236.985965,16039.870000,12.000000


# Data preprocessing Part 2

## Interpolate closest values

In [5]:
# Interpolate missing values based on closest values at the same location
data_ipl = data.copy()

max_range = pd.Timedelta(30, unit="day")
min_range = pd.Timedelta(-30, unit="day")

# List of features with missing values
features = [
 'Chl-a',
 'salinity',
 'T',
 'SECCHI',
 'DIN',
 'PO4-P',
 ]

# Iterate through all features
for fet in features:
    # Iterate over data and select an index and date of the data instance
    for idx, date in data["date"].iteritems():
        # check if the instance has NaN for this feature
        if pd.isna(data[fet][idx]):            
            # l = []
            # Iterate over data again and select an index and date of the data instance
            for idx2, date2 in data["date"].iteritems():
                # Check if the instances are from the same sampling station
                if (data["sampling station"][idx2] == data["sampling station"][idx]):
                    # Calculate the timespan between the two instances
                    timespan = date - date2
                    if min_range <= timespan <= max_range:
                        # l.append(idx2) 
                        data_ipl.loc[idx, fet] = data[fet][idx2]
                    else:
                        continue
                else:
                    continue      
        else:
            continue

In [6]:
data_ipl.isnull().sum()

date                       0
sampling station           0
DSP                        1
DSP_like                  60
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    422
salinity                  21
T                         59
SECCHI                   450
DIN                      352
PO4-P                    349
temp_dif                 346
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

## Prepare original data for ML

In [7]:
# dataframe with selected features
data.drop(columns=["DSP_like", "temp_dif"], inplace=True)#
# data.isnull().sum()

# set date as index for slicing
# data_tree["org_idx"] = df_ipl_allF_L.index
# data_tree.set_index('date', inplace=True)

In [8]:
# Class distribution
data["lipophylic_toxins"].value_counts(dropna=False)

neg    996
NaN    320
poz    136
Name: lipophylic_toxins, dtype: int64

In [9]:
# EXPORT df with all features
# Create pickle
data.to_pickle(f"data/preprocessed/hab_org-data-{nb_fname}")
# Export csv
data.to_csv(f"data/preprocessed/hab_org-data-{nb_fname}.csv", sep=",", na_rep="?", index=False)

## Prepare interpolated data for ML

In [10]:
# All features
# dataframe with selected features
data_ipl.drop(columns=["DSP_like", "temp_dif"], inplace=True)#
data_ipl.isnull().sum()

# set date as index for slicing
# df_ipl_allF_L["org_idx"] = df_ipl_allF_L.index
# df_ipl_allF.set_index('date', inplace=True)

date                       0
sampling station           0
DSP                        1
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    422
salinity                  21
T                         59
SECCHI                   450
DIN                      352
PO4-P                    349
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

In [11]:
# Class distribution
data_ipl["lipophylic_toxins"].value_counts(dropna=False)

neg    996
NaN    320
poz    136
Name: lipophylic_toxins, dtype: int64

In [12]:
# EXPORT interpolated df with all features
# Create pickle
data_ipl.to_pickle(f"data/preprocessed/hab_interp_data-{nb_fname}")
# Export csv
data_ipl.to_csv(f"data/preprocessed/hab_interp_data-{nb_fname}.csv", sep=",", na_rep="?", index=False)